In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import os
import copy
from torch.utils.data import random_split, DataLoader
import gc


In [3]:
# Set the path for the dataset
data_dir = r"C:\Users\venug\Downloads\archive\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\Reduced_Dataset"
classes = ['Cyst', 'Normal', 'Stone', 'Tumor']


In [4]:
# Define data transforms
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load all images from the dataset directory
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

# Split into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size   # 20% for validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create data loaders
batch_size = 16  # Reduced batch size to avoid OOM
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
class_names = full_dataset.classes


In [5]:
# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [6]:
# Function to train and evaluate the model with mixed precision
def train_model(model, criterion, optimizer, scheduler, num_epochs=10, accumulation_steps=2):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    scaler = torch.cuda.amp.GradScaler()  # Mixed precision scaler

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass with mixed precision
                with torch.set_grad_enabled(phase == 'train'):
                    with torch.cuda.amp.autocast():
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                    # Backward pass + optimize in training phase
                    if phase == 'train':
                        scaler.scale(loss).backward()

                        # Gradient accumulation
                        if (i + 1) % accumulation_steps == 0:
                            scaler.step(optimizer)
                            scaler.update()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it has better accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [7]:
# Function to set up and fine-tune a model
def fine_tune_model(architecture_name, num_classes=4):
    if architecture_name == 'resnet18':
        model = models.resnet18(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

    elif architecture_name == 'vgg16':
        model = models.vgg16(pretrained=True)
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, num_classes)

    elif architecture_name == 'mobilenet_v2':
        model = models.mobilenet_v2(pretrained=True)
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)

    elif architecture_name == 'efficientnet_b0':
        model = models.efficientnet_b0(pretrained=True)
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)

    elif architecture_name == 'densenet121':
        model = models.densenet121(pretrained=True)
        num_ftrs = model.classifier.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)

    elif architecture_name == 'alexnet':
        model = models.alexnet(pretrained=True)
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, num_classes)

    elif architecture_name == 'squeezenet':
        model = models.squeezenet1_0(pretrained=True)
        model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))
        model.num_classes = num_classes

    else:
        raise ValueError("Unknown architecture. Choose from 'resnet18', 'vgg16', 'mobilenet_v2', 'efficientnet_b0', 'densenet121', 'alexnet', or 'squeezenet'")

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    return model, criterion, optimizer, exp_lr_scheduler


In [8]:
# Clean up and free any GPU memory before starting
gc.collect()
torch.cuda.empty_cache()


In [11]:
# # Fine-tune VGG16 model
# arch = 'vgg16'
# print(f"Fine-tuning {arch}...")
# model, criterion, optimizer, scheduler = fine_tune_model(arch)
# model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

# # Save the model weights
# torch.save(model.state_dict(), f'{arch}_fine_tuned.pth')
# print(f"Model {arch} saved!\n")


In [10]:
# # Clean up and free any GPU memory before starting next model
del model, optimizer, scheduler
gc.collect()
torch.cuda.empty_cache()


In [9]:
# Fine-tune MobileNetV2 model
arch = 'mobilenet_v2'
print(f"Fine-tuning {arch}...")
model, criterion, optimizer, scheduler = fine_tune_model(arch)
model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

# Save the model weights
torch.save(model.state_dict(), f'{arch}_fine_tuned.pth')
print(f"Model {arch} saved!\n")


Fine-tuning mobilenet_v2...


c:\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/9
----------
train Loss: 0.8058 Acc: 0.6901
val Loss: 0.3490 Acc: 0.8826

Epoch 1/9
----------
train Loss: 0.2672 Acc: 0.9124
val Loss: 0.1237 Acc: 0.9646

Epoch 2/9
----------
train Loss: 0.1472 Acc: 0.9542
val Loss: 0.0852 Acc: 0.9727

Epoch 3/9
----------
train Loss: 0.0900 Acc: 0.9743
val Loss: 0.0425 Acc: 0.9855

Epoch 4/9
----------
train Loss: 0.0620 Acc: 0.9779
val Loss: 0.0408 Acc: 0.9823

Epoch 5/9
----------
train Loss: 0.0508 Acc: 0.9851
val Loss: 0.0362 Acc: 0.9904

Epoch 6/9
----------
train Loss: 0.0329 Acc: 0.9912
val Loss: 0.0362 Acc: 0.9952

Epoch 7/9
----------
train Loss: 0.0341 Acc: 0.9908
val Loss: 0.0201 Acc: 0.9936

Epoch 8/9
----------
train Loss: 0.0257 Acc: 0.9940
val Loss: 0.0304 Acc: 0.9920

Epoch 9/9
----------
train Loss: 0.0187 Acc: 0.9956
val Loss: 0.0245 Acc: 0.9968

Training complete in 23m 51s
Best val Acc: 0.9968
Model mobilenet_v2 saved!



In [9]:
# Fine-tune EfficientNetB0 model
arch = 'efficientnet_b0'
print(f"Fine-tuning {arch}...")
model, criterion, optimizer, scheduler = fine_tune_model(arch)
model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

# Save the model weights
torch.save(model.state_dict(), f'{arch}_fine_tuned.pth')
print(f"Model {arch} saved!\n")


Fine-tuning efficientnet_b0...


c:\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to C:\Users\venug/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:05<00:00, 3.88MB/s]


Epoch 0/9
----------
train Loss: 1.0700 Acc: 0.5715
val Loss: 0.7375 Acc: 0.7203

Epoch 1/9
----------
train Loss: 0.6978 Acc: 0.7267
val Loss: 0.4755 Acc: 0.8489

Epoch 2/9
----------
train Loss: 0.5038 Acc: 0.8400
val Loss: 0.3336 Acc: 0.9035

Epoch 3/9
----------
train Loss: 0.3690 Acc: 0.8766
val Loss: 0.2711 Acc: 0.9196

Epoch 4/9
----------
train Loss: 0.2781 Acc: 0.9168
val Loss: 0.1593 Acc: 0.9550

Epoch 5/9
----------
train Loss: 0.2138 Acc: 0.9429
val Loss: 0.1280 Acc: 0.9711

Epoch 6/9
----------
train Loss: 0.1692 Acc: 0.9542
val Loss: 0.0903 Acc: 0.9791

Epoch 7/9
----------
train Loss: 0.1447 Acc: 0.9634
val Loss: 0.0845 Acc: 0.9807

Epoch 8/9
----------
train Loss: 0.1312 Acc: 0.9691
val Loss: 0.0805 Acc: 0.9807

Epoch 9/9
----------
train Loss: 0.1397 Acc: 0.9682
val Loss: 0.0772 Acc: 0.9839

Training complete in 16m 58s
Best val Acc: 0.9839
Model efficientnet_b0 saved!



In [11]:
# Fine-tune AlexNet model
arch = 'alexnet'
print(f"Fine-tuning {arch}...")
model, criterion, optimizer, scheduler = fine_tune_model(arch)
model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

# Save the model weights
torch.save(model.state_dict(), f'{arch}_fine_tuned.pth')
print(f"Model {arch} saved!\n")


Fine-tuning alexnet...


c:\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\venug/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:52<00:00, 4.69MB/s] 


Epoch 0/9
----------
train Loss: 0.9648 Acc: 0.6234
val Loss: 0.5637 Acc: 0.7974

Epoch 1/9
----------
train Loss: 0.4868 Acc: 0.8123
val Loss: 0.4665 Acc: 0.8087

Epoch 2/9
----------
train Loss: 0.3288 Acc: 0.8754
val Loss: 0.2723 Acc: 0.9035

Epoch 3/9
----------
train Loss: 0.2482 Acc: 0.9092
val Loss: 0.1573 Acc: 0.9534

Epoch 4/9
----------
train Loss: 0.1854 Acc: 0.9309
val Loss: 0.0945 Acc: 0.9727

Epoch 5/9
----------
train Loss: 0.1417 Acc: 0.9486
val Loss: 0.1442 Acc: 0.9421

Epoch 6/9
----------
train Loss: 0.1742 Acc: 0.9345
val Loss: 0.0662 Acc: 0.9791

Epoch 7/9
----------
train Loss: 0.0474 Acc: 0.9867
val Loss: 0.0557 Acc: 0.9839

Epoch 8/9
----------
train Loss: 0.0380 Acc: 0.9879
val Loss: 0.0497 Acc: 0.9871

Epoch 9/9
----------
train Loss: 0.0342 Acc: 0.9936
val Loss: 0.0355 Acc: 0.9920

Training complete in 10m 57s
Best val Acc: 0.9920
Model alexnet saved!



In [9]:
# Fine-tune SqueezeNet model
arch = 'squeezenet'
print(f"Fine-tuning {arch}...")
model, criterion, optimizer, scheduler = fine_tune_model(arch)
model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

# Save the model weights
torch.save(model.state_dict(), f'{arch}_fine_tuned.pth')
print(f"Model {arch} saved!\n")


Fine-tuning squeezenet...


c:\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to C:\Users\venug/.cache\torch\hub\checkpoints\squeezenet1_0-b66bff10.pth
100%|██████████| 4.78M/4.78M [00:02<00:00, 2.34MB/s]


Epoch 0/9
----------
train Loss: 1.1439 Acc: 0.5490
val Loss: 0.8558 Acc: 0.7138

Epoch 1/9
----------
train Loss: 0.7864 Acc: 0.7279
val Loss: 0.6003 Acc: 0.7637

Epoch 2/9
----------
train Loss: 0.4984 Acc: 0.8111
val Loss: 0.3633 Acc: 0.8746

Epoch 3/9
----------
train Loss: 0.4400 Acc: 0.8473
val Loss: 0.3044 Acc: 0.8859

Epoch 4/9
----------
train Loss: 0.3279 Acc: 0.8899
val Loss: 0.2556 Acc: 0.9100

Epoch 5/9
----------
train Loss: 0.2445 Acc: 0.9124
val Loss: 0.2388 Acc: 0.9019

Epoch 6/9
----------
train Loss: 0.1699 Acc: 0.9421
val Loss: 0.6723 Acc: 0.7765

Epoch 7/9
----------
train Loss: 0.1814 Acc: 0.9425
val Loss: 0.1235 Acc: 0.9711

Epoch 8/9
----------
train Loss: 0.0665 Acc: 0.9843
val Loss: 0.1054 Acc: 0.9727

Epoch 9/9
----------
train Loss: 0.0641 Acc: 0.9831
val Loss: 0.0922 Acc: 0.9759

Training complete in 20m 23s
Best val Acc: 0.9759
Model squeezenet saved!



In [ ]:
# Fine-tune DenseNet121 model
arch = 'densenet121'
print(f"Fine-tuning {arch}...")
model, criterion, optimizer, scheduler = fine_tune_model(arch)
model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

# Save the model weights
torch.save(model.state_dict(), f'{arch}_fine_tuned.pth')
print(f"Model {arch} saved!\n")
